In [1]:
import numpy as np

# Regularized gradient descent

In [2]:
class LinearRegressionClass:
    def __init__(self):
        self.coef_ = 0
        self.intercept_ = 0
        
    # Class Methods
    def fit(self, X, y, α=0.1, λ=0, max_iter=10**6, tols=10e-8):
        '''
            Looks for the best coefficients given the data using batch gradient descent.
        '''
        m, n = X.shape # number of rows (data points), number of features
        
        # Random initial guess for the weights and bias
        old_weights = np.random.rand(n) 
        old_bias = 0
        
        # Gradient descent
        for iteration in range(max_iter):
            new_weights = old_weights*(1 - (α*λ/m)) - α*(1/m)*np.dot(X.T, (np.dot(X, old_weights) + old_bias - y))
            new_bias = old_bias - α * (1/m) * np.sum(np.dot(X, old_weights) + old_bias - y)
            
            # Stopping criterion
            if (np.linalg.norm(new_weights - old_weights) > tols) and (np.abs(new_bias - old_bias) > tols):
                old_weights = new_weights
                old_bias = new_bias

            else:
                print("Converged after {} iterations.".format(iteration))
                # Best Model Coefficients
                self.coef_ = new_weights 
                self.intercept_ = new_bias
                
                break

        # Compute for the linear regression model's score
        predicted_y = np.array([np.dot(Xi, self.coef_) + self.intercept_ for Xi in X]) # predicted y for the X[i] data

        SS_res = np.sum((y - predicted_y)**2)    
        SS_tot = np.sum((y - y.mean())**2)
        self.R2 = 1 - (SS_res/SS_tot) # Model Score
    
    
    def predict(self, x_test):
        #x = np.insert(x_test, 0, 1)
        return np.dot(x_test, self.coef_) + self.intercept_

# Make data

In [3]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=500, n_features=20, noise=20)

### Fit and Predict

In [4]:
# Feed the data to the linear regression model with random initial parameters
regtest = LinearRegressionClass() 
regtest.fit(X, y, α=0.01, λ=50) # Learn from data

# Print the coefficients and model score
print(regtest.coef_, regtest.R2)

Converged after 1810 iterations.
[59.5500053  32.19848128  0.10775547 -1.56023012  1.36953462  0.75969074
 -0.66838078 37.61628656 47.41328549 -0.33045666 85.42280585 31.21587149
 -1.23863274 -1.90780094 -0.64094315 68.71275511 -0.20983064 23.71496193
 44.97961748 27.22345266] 0.9784710429846897


### Make prediction

In [5]:
# Make a prediction
test_case = np.random.rand(len(X[0]))
regtest.predict(test_case)

169.50225407319337

### Scikit-learn

In [6]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X, y)
print(reg.intercept_, reg.coef_, reg.score(X, y))

0.5837087690044243 [ 6.64309519e+01  3.53182098e+01  3.79237774e-01 -7.50490762e-01
  9.36064064e-01  1.84373479e-03 -9.99875278e-01  4.22218262e+01
  5.10919393e+01  2.72044306e-01  9.39258749e+01  3.42394763e+01
 -1.81352261e+00 -1.57563099e+00 -3.92429934e-01  7.68454916e+01
  4.09069811e-02  2.63042804e+01  4.97512146e+01  3.05793983e+01] 0.9873123409913249


In [7]:
reg.predict(np.array([test_case]))[0]

185.8323844326846

# Regularized normal equation

In [8]:
class LinearRegressionNormalEquation_Regularization:
    def __init__(self):
        self.coef_ = None
        
    # Class Methods
    def fit(self, X, y, λ=0):
        '''
        Input:
            X = m x n array. The features matrix.
            y = m x 1 array. The target variable.        
        '''
        m, n = X.shape
        X = np.vstack((np.ones(len(X)), X.T)).T # column of ones in the first column
        
        # Normal equation
        self.coef_ = np.linalg.inv((X.T @ X) + λ*np.diag(np.insert(np.ones(n),0, 0))) @ X.T @ y
        
        # Compute for the linear regression model's score
        predicted_y = np.array([self.coef_ @ Xi for Xi in X])
        
        SS_res = 0
        for i in range(m):
            SS_res += (y[i] - predicted_y[i])**2
            
        SS_tot = 0
        for i in range(m):
            SS_tot += (y[i] - y.mean())**2
        
        self.R2 = 1 - (SS_res/SS_tot)
        
    
    def predict(self, x_test):
        x_test = np.array([np.insert(x_test[i], 0, 1) for i in range(len(x_test))])
        return np.array([self.coef_ @ x_test_i for x_test_i in x_test])

### Test

In [9]:
# Feed the data to the linear regression model with random initial parameters
regtestnormal = LinearRegressionNormalEquation_Regularization() 
regtestnormal.fit(X, y, λ=100) # Learn from data

In [10]:
regtestnormal.R2

0.9579536688084703